In [1]:
# Run in terminal or command prompt
# python3 -m spacy download en

import numpy as np
import pandas as pd
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet as wn
import re, nltk, spacy, gensim, string

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\driss\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# empty doc list
doc_complete = []

with open (".\\data\\stackoverflow_vacature_tekst2.csv", "r") as myfile:
    for line in myfile:
        doc_complete.append(line)

In [3]:
tech_words_set = set()

with open(".\\data\\top50000_tags_short2.csv", "r") as myfiletags:
    for line in myfiletags:
        #tech_words_set.add(line.replace('"',''))
        tech_words_set.add(line.rstrip())
        
#for word in tech_words_set:
#    print(word)


In [4]:
stop = set(stopwords.words(['english', 'german', 'dutch']))
#stop_de = set(stopwords.words('german'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    #stop_de_free = " ".join([i for i in stop_free.lower().split() if i not in stop_de])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    #print(normalized)
    #[print(i) for i in normalized.lower().split()]
    normalized2 = " ".join([i for i in normalized.lower().split() if i in tech_words_set])
    #print(normalized2)
    return normalized2
    ####return normalized

doc_clean = [clean(doc).split() for doc in doc_complete] 

In [5]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(doc_clean, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [6]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words=['english','german'],  # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{1,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [7]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  4.97809147374 %


In [8]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(doc_clean, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
# print(data_lemmatized[:2])

In [9]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=25,           # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

# print(lda_model)  # Model attributes

In [10]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
# pprint(lda_model.get_params())

Log Likelihood:  -305824.100183
Perplexity:  105.752357036


In [11]:
# Define Search Param
search_params = {'n_components': [22], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params,cv=5)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_components': [22], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [12]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.7, 'n_components': 22}
Best Log Likelihood Score:  -66774.846764
Model Perplexity:  100.515275019


In [13]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ['Topic' + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(doc_clean))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(100).style.applymap(color_green).applymap(make_bold)
df_document_topics

In [14]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,17,353
1,13,270
2,3,231
3,11,194
4,6,163
5,0,161
6,9,148
7,1,145
8,4,145
9,2,121


In [15]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
17     10.165636        1       1   63.100574  154.573120
1       8.133156        1       2 -140.926559   23.401974
13      7.549128        1       3  152.949371  -66.169449
3       6.833955        1       4   75.275063   84.961983
11      5.952557        1       5   22.649389  -74.868973
9       5.439320        1       6  -17.610155   -0.189857
4       5.379138        1       7   26.700384   44.123028
6       4.540561        1       8   90.531937  -44.584648
8       4.487440        1       9   90.731827   22.038177
21      4.440788        1      10  -36.377716   62.171169
0       4.419760        1      11   10.538663 -140.638992
2       4.173232        1      12  -76.824615    8.044919
14      3.916366        1      13   84.907791 -115.742455
12      3.812533        1      14   37.594364  -16.156477
7       3.612991        1      15 -111.539169  -55.866611
16      3.213898        1      16  142.019897   88.683174
10      3.108850        1      17  -38.913952  -56.983845
19      2.531917        1      18  156.974670   11.992506
5       2.165018        1      19  -63.507126 -119.846970
20      2.127826        1      20  -97.315491   89.016289
15      2.007051        1      21   10.636537  107.535355
18      1.988879        1      22  -44.660637  144.866364, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
43    Default  1599.000000        cloud  1599.000000  30.0000  30.0000
252   Default  1401.000000     security  1401.000000  29.0000  29.0000
172   Default  1025.000000       mobile  1025.000000  28.0000  28.0000
32    Default   818.000000            c   818.000000  27.0000  27.0000
128   Default  1503.000000         java  1503.000000  26.0000  26.0000
22    Default   866.000000   automation   866.000000  25.0000  25.0000
282   Default  1642.000000      testing  1642.000000  24.0000  24.0000
30    Default  2228.000000        build  2228.000000  23.0000  23.0000
296   Default   482.000000        video   482.000000  22.0000  22.0000
204   Default   448.000000          php   448.000000  21.0000  21.0000
129   Default  1194.000000   javascript  1194.000000  20.0000  20.0000
209   Default  1178.000000     position  1178.000000  19.0000  19.0000
41    Default  1016.000000       client  1016.000000  18.0000  18.0000
57    Default  1054.000000     database  1054.000000  17.0000  17.0000
299   Default  2169.000000          web  2169.000000  16.0000  16.0000
64    Default  4097.000000       design  4097.000000  15.0000  15.0000
213   Default  1821.000000      process  1821.000000  14.0000  14.0000
6     Default   442.000000      android   442.000000  13.0000  13.0000
217   Default  1024.000000       python  1024.000000  12.0000  12.0000
257   Default  3206.000000      service  3206.000000  11.0000  11.0000
256   Default   684.000000       server   684.000000  10.0000  10.0000
267   Default   797.000000          sql   797.000000   9.0000   9.0000
266   Default   330.000000       spring   330.000000   8.0000   8.0000
202   Default  1234.000000  performance  1234.000000   7.0000   7.0000
4     Default   329.000000    algorithm   329.000000   6.0000   6.0000
175   Default   414.000000        model   414.000000   5.0000   5.0000
23    Default   282.000000        azure   282.000000   4.0000   4.0000
98    Default   402.000000           go   402.000000   3.0000   3.0000
302   Default   238.000000       window   238.000000   2.0000   2.0000
151   Default   619.000000        linux   619.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
279   Topic22    27.756429   tensorflow    43.845278   3.4604  -3.8504
32    Topic22   431.354861            c   818.161575   3.2775  -1.1070
31    Topic22    21.345938       button    42.727707   3.2236  -4.1131
231   Top

In [16]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
# df_topic_keywords.head()

,3d,air,ajax,akka,algorithm,alignment,android,angular,angularj,angularjs,...,wcf,web,webdriver,webpack,window,wordpress,wpf,xamarin,xcode,xml
Topic0,0.045455,0.045455,0.045455,15.929407,0.045455,0.045455,0.045455,0.045455,1.784216,0.045455,...,0.045455,58.530942,0.045455,0.045455,0.045455,0.045455,0.045455,0.047456,0.045455,0.045455
Topic1,0.045455,0.066210,0.045455,0.045455,0.045455,7.914453,0.045455,0.045455,25.140809,0.045455,...,0.045455,79.083408,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455
Topic2,0.045455,0.045455,0.045455,0.045455,0.068959,0.045455,0.045455,0.045455,0.045455,0.045455,...,0.045455,7.650301,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455
Topic3,0.045455,1.435583,0.045455,0.045455,0.045455,43.827853,0.045455,0.045455,0.045455,0.045455,...,0.045455,65.518039,0.045455,0.045455,0.048809,0.045455,7.465905,0.045455,0.045455,6.311866
Topic4,0.045455,7.645911,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,...,0.045455,83.283900,0.045455,0.045455,7.809678,0.045455,0.045455,0.045455,0.045455,0.045455


In [17]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,go,docker,get,mongodb,python,join,scala,find,class,process,salesforce,path,build,kubernete,stack
Topic 1,security,design,service,architecture,performance,time,database,testing,process,key,position,build,cloud,automation,view
Topic 2,build,python,scripting,jenkin,process,service,performance,automation,docker,shell,testing,linux,maven,deployment,git
Topic 3,client,time,process,get,position,build,service,join,key,background,focus,report,find,scope,include
Topic 4,cloud,service,architecture,python,deployment,get,azure,web,build,arm,join,docker,design,linux,stack
Topic 5,c,3d,command,perl,sql,stack,scripting,client,powershell,python,background,server,build,git,linux
Topic 6,mobile,android,design,testing,swift,build,java,sdk,join,kotlin,git,service,focus,api,animation
Topic 7,java,spring,service,hibernate,xml,oracle,rest,design,junit,client,json,apache,git,web,sql
Topic 8,design,api,rest,package,performance,service,stack,architecture,web,testing,javascript,redux,position,java,function
Topic 9,database,web,sql,server,design,oracle,service,nosql,time,postgresql,stack,python,join,mysql,testing
